## Preface
In the project, we extract data from amazon, and prepare for further analysis. Also, we try to visualize data in order to describe consumptional behaviors.


## Collect data
We use BeautifulSoup to parse the raw HTML. The variables for each items are: image data in bytes, image link, item link, name of goods, numbers of rating, price and rating(0-5).

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd

# one page item
def getOnePage(url):
    onepage = []
    count = 0
    getting = False
    while not getting:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'
            }

        # call response
        response = requests.get(url, headers=headers)

        # build Soup object 
        root = BeautifulSoup(response.text, 'html5lib')

        # get a list contain each item's HTML
        # try serveal times until it is correct
    
        print(getting)
        items = root.find_all("div", class_="s-item-container")
        count = count + 1
        if len(items) != 0:
            getting = True    
            
    # collect each item in one page
    for i in items:
        try:
            # deal with each item HTML
            d = {}
            d['img_data'] = requests.get(i.find("img")['src']).content
            d['img_link'] = i.find("img")['src']
            d['link'] = i.find("a", class_="s-access-detail-page")['href']
            d['name'] = i.find("a", class_="s-access-detail-page").text
            d['num_ratings'] = \
            float((i.find("a", {"href":lambda x : x.endswith("#customerReviews")}).text).replace(',',''))
            d['price'] = \
            float(i.find("span", class_="sx-price-whole").text + '.'+\
            i.find("sup",class_="sx-price-fractional").text
            )
            d['stars'] = i.find("i",class_="a-icon-star").text
            # print(d)
            onepage.append(d)
        except (TypeError, AttributeError):
            pass
        
    return onepage

    
# url = 'https://www.amazon.com/s/ref=sd_allcat_nav_desktop_sa_intl_baby_toddler_toys?_encoding=UTF8&bbn=16225005011&rh=i%3Aspecialty-aps%2Cn%3A%2116225005011%2Cn%3A196601011'
# getOnePage(url)

In [3]:
def getUrl(page):
    return 'https://www.amazon.com/s/ref=sr_pg_'+str(page)+\
    '?rh=i%3Ababy-products-intl-ship%2Cn%3A%2116225005011%2Cn%3A196601011&page='+str(page)+\
    '&bbn=16225005011&ie=UTF8&qid=1549443515'

In [ ]:
# all scapling
def scaplingAll():
    """return a dataframe"""
    #df = pd.DataFrame()
    df = pd.read_csv('amazon_toys.csv')
    page = 48
    while True and page<400:
        # get url
        url = getUrl(page)
        print(url)
        
        # get one page dataframe
        onePageItems = getOnePage(url)
        print('this page has data:', len(onePageItems))
        df = df.append(onePageItems) # add dataframe
        print('have already been in page: ', page)
        print('----------------------------')
        page = page + 1
        
        # save df
        df.to_csv('amazon_toys.csv', encoding='utf-8', index=False)
    

    return df


items = scaplingAll()



In [11]:

df = pd.read_csv('amazon_toys.csv')
df.head()

,img_data,img_link,link,name,num_ratings,price,stars
0,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Nuby-Ice-Gel-Teether-Ke...,Nuby Ice Gel Teether Keys,3184.0,3.54,4.3 out of 5 stars
1,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/First-Years-Stack-Up-Cu...,The First Years Stack Up Cups,2811.0,3.99,4.8 out of 5 stars
2,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Baby-Banana-Training-To...,Baby Banana Infant Training Toothbrush and Tee...,7656.0,6.99,4.7 out of 5 stars
3,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Mega-Bloks-80-Piece-Bui...,"Mega Bloks 80-Piece Big Building Bag, Classic",5954.0,14.92,4.7 out of 5 stars
4,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Baby-Einstein-Along-Tun...,Baby Einstein Take Along Tunes Musical Toy,10217.0,8.99,4.6 out of 5 stars


In [12]:
df.shape

(6271, 7)

Now, we get the final data frame object using for further analysis.